In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, RDFS, DCTERMS #most common namespaces
import urllib.parse #for parsing strings to URI's
import numpy as np
from array import array

Define Namespaces for the Data

In [2]:
g = Graph()
title = Namespace('http://example.org/title/')
author = Namespace('http://example.org/author/')
year = Namespace('http://example.org/year/')
doi = Namespace('http://example.org/doi/')
pmid = Namespace('http://example.org/PMID/')
journal = Namespace('http://example.org/journal/')
volume = Namespace('http://example.org/volume/')
abstract = Namespace('http://example.org/abstract/')
keyword_abstract = Namespace('http://example.org/keyword_abstract/')
keyword_mesh = Namespace('http://example.org/keywords_mesh/')
keyword_merged = Namespace('http://example.org/keywords_merged/')
links = Namespace('http://example.org/fulltext_links/')
articles = Namespace('http://example.org/related_articles/')
fields = Namespace('http://example.org/fields/')
subfields = Namespace('http://example.org/subfield/')
aimethod = Namespace('http://example.org/ai_method')

Upload the data files and merge new examples with diseases

In [3]:
new_example = pd.read_json('example_new.json')
diseases = pd.read_csv('mesh_papers_with_disease_n_abbr.csv')
diseases['PMID']= diseases['PMID'].replace('PMID:', '',regex=True)
new_example=new_example.merge(diseases, on='PMID', how='right')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
algorithms = pd.read_csv("Book.csv", sep=';', header=0 )
algorithms.head(30)

In [ ]:
regression = algorithms['regression'].values
instance_based = algorithms['instance based'].values
#regularization = algorithms['regularization'].values
decision_tree = algorithms['decision tree'].values
bayesian = algorithms['bayesian'].values
clustering = algorithms['clustering'].values
association_rule = algorithms['association rule learning'].values
ann = algorithms['artificial neural network'].values
deep_learning = algorithms['deepl learning'].values
dimensionality_reduction = algorithms['dimensionality reduction'].values
ensemble_algorithm = algorithms['ensemble algorithm'].values
regularization = ['Ridge Regression','Least Absolute Shrinkage and Selection Operator','Elastic Net','Least-Angle Regression','LASSO','LARS']

In [105]:
indentation = []
indentation.append(0)
depth = 0

f = open("acm_hierarchy_restructure2.txt", 'r')

for line in f:
    line = line[:-1]

    content = line.strip()
    indent = len(line) - len(content)
    if indent > indentation[-1]:
        depth += 1
        indentation.append(indent)
        data.append([])

    elif indent < indentation[-1]:
        while indent < indentation[-1]:
            depth -= 1
            indentation.pop()
            top = data.pop()
            data[-1].append(top)

        #if indent != indentation[-3]:
        #    raise RuntimeError("Bad formatting")

    data[-1].append(content)

while len(data) > 1:
    top = data.pop()
    data[-1].append(top)

In [107]:
data

[[[[]],
  [],
  [[], []],
  [[[]]],
  [[[], []], [[], [], []]],
  [],
  [[[]], [[[]], [[]], [[], [], [[]]], [[[]]]]],
  [],
  [[], [], [], [[]], [[]], [], []],
  [[], [], [[[]]]],
  'Artificial intelligence | artificial intelligence, ai ,computational intelligence,algorithm,autonomous,automated,automatic,computer aided',
  ['Natural language processing | natural language processing, nlp',
   ['Syntactic analysis |',
    'Grammar induction | grammar induction,grammar learning',
    ['Stemming |',
     'Lemmatization |',
     'Morphological segmentation |',
     'Part-of-speech tagging | part of speech tagging,pos tagging',
     'Syntactic Parsing |',
     'Sentence boundary disambiguation | sentence boundary disambiguation,sentence breaking',
     'Word segmentation |',
     'Terminology extraction |',
     'Semantic analysis |',
     'Lexical semantics |',
     'Distributional semantics |',
     'Machine Translation |',
     'Named entity recognition |  ner ,entity recognition',
     '

In [109]:
method = [[[x for col in range(a)] for col in range(b)] for row in range(c)]

NameError: name 'c' is not defined

Start building the RDF Schema

In [7]:
for index, row in new_example.iterrows():
    
    authors = str(row['Authors']).split(";")
    for a in authors[:-1]:
        g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['creator'], Literal(a, datatype=XSD.string) ))
        
    keywords = str(row['Keywords_Merged']).split(";")
    for key in keywords:
        g.add((URIRef(pmid+str(row['PMID'])), URIRef(keyword_merged+'key_merged'), Literal(key, datatype=XSD.string) ))
    
    fullTexts = str(row['FullTextLinks']).split(";")
    for text in fullTexts:
        g.add((URIRef(pmid+str(row['PMID'])), RDFS.comment, Literal(text, datatype=XSD.string) ))
        
    related = str(row['FullTextLinks']).split(";")
    for a in related:
        g.add((URIRef(pmid+str(row['PMID'])), RDFS.seeAlso, Literal(a, datatype=XSD.string) ))
    
    g.add((URIRef(pmid+str(row['PMID'])),RDF.type, Literal(row['PMID'])))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['identifier'], Literal(row['PMID']))) 
    g.add((URIRef(doi+str(row['PMID'])), DCTERMS['identifier'],Literal(row['Doi'], datatype=XSD.integer) ))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['publisher'], Literal(row['Journal'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+str(row['PMID'])), URIRef(volume+'volume'), Literal(row['Volume'], datatype=XSD.string) ))
    g.add((URIRef(pmid+str(row['PMID'])), DCTERMS['abstract'], Literal(row['Abstract'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+row['PMID']), URIRef(field+'field'), Literal(row['Field'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+row['PMID']), RDFS.subclassOf, Literal(row['Subfield'], datatype=XSD.string) ))
    #g.add((URIRef(pmid+str(row['PMID'])), URIRef(aimethod), Literal(row['Methods'])))
    #g.add((URIRef(loc+urllib.parse.quote(row['Address'])), URIRef(schema+'name'), Literal(row['Address'], datatype=XSD.string) ))
    
    

In [8]:
print(g.serialize(format='turtle').decode('UTF-8'))

@prefix ns1: <http://purl.org/dc/terms/> .
@prefix ns2: <http://example.org/keywords_merged/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/PMID/24694170> a 24694170 ;
    ns2:key_merged "  "^^xsd:string,
        "  Artificial Intelligence*"^^xsd:string,
        "  Electroencephalography / classification*"^^xsd:string,
        "  Electroencephalography / methods*"^^xsd:string,
        "  Epilepsy / diagnosis*"^^xsd:string,
        "  Humans"^^xsd:string,
        "  Research Support, Non-U.S. Gov't"^^xsd:string,
        "  Signal Processing, Computer-Assisted*"^^xsd:string,
        "  Support Vector Machine"^^xsd:string ;
    ns1:abstract " The automatic identification of epileptic EEG signals is significant in both relieving heavy workload of visual inspection of EEG recordings and treatment of epilepsy. This paper presents a novel method based on the theory of sparse representation to identify epilepti

Querying RDF via SPARQL

In [ ]:
q = "select ?title ?abstract where {?title ?abstract <http://example.org/PMID/24694170>}"
x = g.query(q)
t = list(x)
print("----------------------------------------------------------------------------------------------")
for y in range(0,len(t)):
    ss=t[y][0]
    #search for id
    q = "select ?relation ?part  where {<"+ss+"> ?relation ?part}"
    xx = g.query(q)
    print(xx)

In [ ]:
# <id, attribute, value>
#search PMID in the searh bar, and return the data with this PMID, and show its attribute and value, for example below PMID=31946810，and have 12 attributes and values
#<a,?,?>
q = "select ?attribute ?value where {<http://example.org/PMID/31946810>?attribute ?value}"
x = g.query(q)
t = list(x)
print(len(t))  
  
for i in t:
    print(i)
    print("----------------------------------------------------------------------------------------------")

In [ ]:
# <id, attribute, value>
#search relationship/attribute title in the searh bar, you can change the other attribute
#we can see all the dataset is 79 data which have title
# <?,b,?>
q = "select ?id ?value where {?id <http://example.org/title/title> ?value}"  
x = g.query(q)
t = list(x)
print(len(t))

for i in t:
    print(i)
    print("----------------------------------------------------------------------------------------------")

In [ ]:
# search value of title in the search bar, for example, search title value is 29993442, but can only do the exactly search
# <id, attribute, value>
# <?,?,c>
q = "select * where {?id <http://example.org/title/title> 29993442 }"  
x = g.query(q)
t = list(x)
print(len(t))   # only one data match this title value
print(t[0])

for i in range(0,len(t)):
    value = str(t[i]).replace("(rdflib.term.URIRef('","").replace("'),)","")  #extract the PMID URI, and then from PMID get all the data
    print(value)
    q = "select * where {<"+ value +"> ?attribute ?value}"  
    x = g.query(q)
    b = list(x)
    if (len(b) > 0) :
        for y in range(0,len(b)):
            print(str(b[y]))
            print("new attibute-------------------------------------------------------------")
        print("\n")
        print("new data-------------------------------------------")

In [ ]:
pip install sparqlwrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper
# maybe something can use in this package, but I don't use this below 

In [ ]:
# Above we can only do the exactly search, which can only get all the matched data with the exact vaule we know
#but we can not do the fuzzy query, and below try the fuzzy query
#when search 31 in the search bar, what will happen?
# <id, attribute, value>
q = "select ?id where {?id ?attribute ?value filter regex(?value, '31')}"
x = g.query(q)
t = list(x)
for i in range(0,len(t)):
    value = str(t[i]).replace("(rdflib.term.URIRef('","").replace("'),)","")
    q = "select * where {<"+ value +"> ?attribute ?value}"          # also get the PMID first and then get all the matched data
    x = g.query(q)
    b = list(x)
    if (len(b) > 0) :
        for y in range(0,len(b)):
            print(str(b[y]))
            print("new attibute-------------------------------------------------------------")
        print("\n")
        print("new data-------------------------------------------")

#we can see that as long as "31" appeare in the value, no matter what attibute, it will match.
#but when I want to search for the exact attibute 'title' which may have something like "**31**", I failed, I can only do with all attibutes.

In [ ]:
# and maybe you can change the title to other attibutes and also try the fuzzy query.